# `Jai Chaudhry 2K18-SE-069`

## Experiment - 7

- Write a program to implement the Naïve Bayesian classifier for appropriate dataset and compute
  the performance measures of the model.

In [22]:
import pandas as pd
import numpy as np
from numpy import log2 as log
import copy
eps = np.finfo(float).eps

print(eps)


2.220446049250313e-16


# Load Data

In [30]:
data = pd.read_csv('C:\\Users\\Jai\\Desktop\\adult.csv',na_values='?')
data = data.dropna() 

print(len(data))
print(data.shape)
data.head(3)


45222
(45222, 15)


age  workclass  fnlwgt   education  educational-num      marital-status  \
0   25    Private  226802        11th                7       Never-married   
1   38    Private   89814     HS-grad                9  Married-civ-spouse   
2   28  Local-gov  336951  Assoc-acdm               12  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   
1    Farming-fishing      Husband  White   Male             0             0   
2    Protective-serv      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              50  United-States  <=50K  
2              40  United-States   >50K

# Processing : Numerical Columns to Categorical 

In [31]:
num_cols = list(data.select_dtypes(include=["number"]).columns)
cat_cols = list(data.select_dtypes(exclude=["number"]).columns)

print("Numeric Cols:\n",num_cols,"\n")
print("Categorical Cols:\n",cat_cols,"\n")


Numeric Cols:
 ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week'] 

Categorical Cols:
 ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income'] 



In [33]:
print(data.shape)

for c in num_cols:
    temp = copy.deepcopy(data[c])
    avg = data[c].mean()
    temp[data[c]<avg] = 0
    temp[data[c]>avg] = 1

    data[c] = temp
print(data.shape)

(45222, 15)
(45222, 15)


In [34]:
print(data.shape)
data.describe()


(45222, 15)


age        fnlwgt  educational-num  capital-gain  \
count  45222.000000  45222.000000     45222.000000  45222.000000   
mean       0.464022      0.437752         0.329021      0.081553   
std        0.498709      0.496115         0.469863      0.273686   
min        0.000000      0.000000         0.000000      0.000000   
25%        0.000000      0.000000         0.000000      0.000000   
50%        0.000000      0.000000         0.000000      0.000000   
75%        1.000000      1.000000         1.000000      0.000000   
max        1.000000      1.000000         1.000000      1.000000   

       capital-loss  hours-per-week  
count  45222.000000    45222.000000  
mean       0.047322        0.304653  
std        0.212329        0.460265  
min        0.000000        0.000000  
25%        0.000000        0.000000  
50%        0.000000        0.000000  
75%        0.000000        1.000000  
max        1.000000        1.000000

# Naive Bayes


In [155]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def find(df,query):
    Class = df.keys()[-1]
    attributes = df.columns
    
    target_variables = df[Class].unique()
    
    probabilities = []
    
    total = len(df)
    
    for target_variable in target_variables:
        
        # get all the instances of this target / output variable
        den = len(df[Class][df[Class]==target_variable])
        
        prob = den/total
        
        for i in range(len(query)):
            attr = attributes[i]
            num = len(df[attr][df[attr]==query[attr]][df[Class]==target_variable])
            
            fraction = num / den
            
            prob *= fraction
            
        probabilities.append(prob)
    return target_variables[np.argmax(probabilities)]
    
    
def print_metrics(preds,actual):
    
    # first converting to 0 / 1
    predictions = np.zeros(len(preds))
    y_actual = np.zeros(len(actual))
    
    print(len(actual))
    
    for i in range(0,len(actual)):
        if preds[i]=='>50K':
            predictions[i] = 1
            
        if actual[i]=='>50K':
            y_actual[i] = 1
    
#     print(y_actual)
#     print(predictions)
    
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_actual,predictions)*100
    print('Accuracy      : %f' % (accuracy),"%")

    # precision tp / (tp + fp)
    precision = precision_score(y_actual,predictions)*100
    print('Precision     : %f' % precision,"%")

    # recall: tp / (tp + fn)
    recall = recall_score(y_actual,predictions)*100
    print('Recall        : %f' % recall,"%")

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_actual,predictions)*100
    print('F1 score      : %f' % f1,"%")

    # ROC AUC
    auc = roc_auc_score(y_actual,predictions)*100
    print("roc_auc_score :",auc,"%")
    

def NaiveBayes(df_train, df_test, samples, printResult = False):
    df_train = df_train.reset_index(drop = True)
    df_test = df_test.reset_index(drop = True)
    
    correct_preds = 0
    
    y_actual = np.array(df_test.iloc[:min(samples,len(df_test)),-1])
        
    preds = []
    
    for i in range(0,min(samples,len(df_test))):
        print(i)
        query = df_test.iloc[i,:df_test.shape[1]-1]
        res = find(df_train,query)
        actual = df_test.iloc[i,:][-1]
        
        preds.append(res)
        
        if res == actual:
            correct_preds += 1
            
        if printResult:
            print("predicted : ", res ," actual: ", actual,correct_preds)
    
    print_metrics(preds,y_actual)
#     acc = (correct_preds/min(samples,len(df_test))) * 100
#     print("acccuracy : ",acc)


# Dataset Splitting

In [156]:
test_size = 0.3
tot_size = len(data)
split = int(tot_size*(1-test_size))

df_train = data[0:split]
df_test = data[split:]

print(len(df_train),len(df_test))


31655 13567


In [157]:
print(df_test.iloc[:,-1].shape)


(13567,)


# Prediction and Performance Evaluation

In [158]:
NaiveBayes(df_train,df_test,len(df_test))


13567
Accuracy      : 82.214196 %
Precision     : 62.824134 %
Recall        : 71.717468 %
F1 score      : 66.976871 %
roc_auc_score : 78.72924101976227 %


# `Jai Chaudhry 2K18-SE-069`